## Model Training - Max

## Part 7: Improved Model Architecture

In [91]:
def create_improved_model(input_shape, output_shape):
    input_layer = Input(shape=input_shape)
    x = Bidirectional(LSTM(units=64, return_sequences=True))(input_layer)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    x = Bidirectional(LSTM(units=64, return_sequences=False))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = Dense(32, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.1)(x)
    output_layer = Dense(output_shape, activation='softmax')(x)
    model = keras.Model(inputs=input_layer, outputs=output_layer)
    opt = keras.optimizers.Adam(learning_rate=0.01)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

input_shape = (X_train_scaled.shape[1], X_train_scaled.shape[2])
output_shape = y_train_categorical.shape[1]
improved_model = create_improved_model(input_shape, output_shape)
improved_model.summary()

Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_6 (InputLayer)           │ (None, 10, 17)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_12 (Bidirectional)     │ (None, 10, 128)             │          41,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_18               │ (None, 10, 128)             │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_18 (Dropout)                 │ (None, 10, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_13 (Bidirectional)     │ (None, 128)                 │          98,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_19               │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_19 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 32)                  │           4,128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_20               │ (None, 32)                  │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_20 (Dropout)                 │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 3)                   │              99 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 146,179 (571.01 KB)

 Trainable params: 145,603 (568.76 KB)

 Non-trainable params: 576 (2.25 KB)

## Part 8: Callbacks for Better Training

In [92]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.0001, verbose=1)
checkpoint = ModelCheckpoint('best_crypto_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)

## Part 9: Model Training

In [ ]:
# Model Training
history = improved_model.fit(
    X_train_scaled, y_train_categorical,
    epochs=20, batch_size=64,
    validation_data=(X_val_scaled, y_val_categorical),
    callbacks=[early_stopping, reduce_lr, checkpoint],
    class_weight=class_weight_dict,
    verbose=1
)

Epoch 1/20
53/55 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.5413 - loss: 1.0167
Epoch 1: val_accuracy improved from -inf to 0.47580, saving model to best_crypto_model.h5


55/55 ━━━━━━━━━━━━━━━━━━━━ 8s 33ms/step - accuracy: 0.5458 - loss: 1.0088 - val_accuracy: 0.4758 - val_loss: 0.9248 - learning_rate: 0.0100
Epoch 2/20
53/55 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7136 - loss: 0.7134
Epoch 2: val_accuracy improved from 0.47580 to 0.66181, saving model to best_crypto_model.h5


55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.7140 - loss: 0.7131 - val_accuracy: 0.6618 - val_loss: 0.8100 - learning_rate: 0.0100
Epoch 3/20
53/55 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7182 - loss: 0.6899
Epoch 3: val_accuracy did not improve from 0.66181
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.7186 - loss: 0.6894 - val_accuracy: 0.6198 - val_loss: 0.8107 - learning_rate: 0.0100
Epoch 4/20
52/55 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7325 - loss: 0.6586
Epoch 4: val_accuracy improved from 0.66181 to 0.72128, saving model to best_crypto_model.h5


55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.7316 - loss: 0.6599 - val_accuracy: 0.7213 - val_loss: 0.6548 - learning_rate: 0.0100
Epoch 5/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7299 - loss: 0.6782
Epoch 5: val_accuracy improved from 0.72128 to 0.75977, saving model to best_crypto_model.h5


55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.7301 - loss: 0.6779 - val_accuracy: 0.7598 - val_loss: 0.6422 - learning_rate: 0.0100
Epoch 6/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.7258 - loss: 0.6605
Epoch 6: val_accuracy did not improve from 0.75977
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.7258 - loss: 0.6605 - val_accuracy: 0.7294 - val_loss: 0.6918 - learning_rate: 0.0100
Epoch 7/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.7343 - loss: 0.6478
Epoch 7: val_accuracy improved from 0.75977 to 0.77784, saving model to best_crypto_model.h5


55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.7343 - loss: 0.6479 - val_accuracy: 0.7778 - val_loss: 0.5903 - learning_rate: 0.0100
Epoch 8/20
54/55 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7439 - loss: 0.6203
Epoch 8: val_accuracy did not improve from 0.77784
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.7435 - loss: 0.6212 - val_accuracy: 0.6933 - val_loss: 0.6940 - learning_rate: 0.0100
Epoch 9/20
52/55 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7584 - loss: 0.6178
Epoch 9: val_accuracy did not improve from 0.77784
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.7577 - loss: 0.6185 - val_accuracy: 0.7708 - val_loss: 0.6114 - learning_rate: 0.0100
Epoch 10/20
54/55 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7574 - loss: 0.6054
Epoch 10: val_accuracy improved from 0.77784 to 0.79883, saving model to best_crypto_model.h5


55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.7571 - loss: 0.6060 - val_accuracy: 0.7988 - val_loss: 0.5785 - learning_rate: 0.0100
Epoch 11/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7511 - loss: 0.5987
Epoch 11: val_accuracy did not improve from 0.79883
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.7511 - loss: 0.5988 - val_accuracy: 0.7254 - val_loss: 0.6653 - learning_rate: 0.0100
Epoch 12/20
52/55 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7475 - loss: 0.6188
Epoch 12: val_accuracy improved from 0.79883 to 0.80991, saving model to best_crypto_model.h5


55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.7482 - loss: 0.6177 - val_accuracy: 0.8099 - val_loss: 0.5183 - learning_rate: 0.0100
Epoch 13/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7665 - loss: 0.5928
Epoch 13: val_accuracy did not improve from 0.80991
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.7664 - loss: 0.5930 - val_accuracy: 0.7534 - val_loss: 0.6567 - learning_rate: 0.0100
Epoch 14/20
51/55 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7558 - loss: 0.6012
Epoch 14: val_accuracy did not improve from 0.80991
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.7565 - loss: 0.6007 - val_accuracy: 0.7335 - val_loss: 0.6307 - learning_rate: 0.0100
Epoch 15/20
54/55 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7635 - loss: 0.5993
Epoch 15: ReduceLROnPlateau reducing learning rate to 0.004999999888241291.

Epoch 15: val_accuracy did not improve from 0.80991
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.7633 - loss: 0.5993 - val_accuracy: